Ficha 3 - Integração e Diferenciação Numérica

In [28]:
from numpy import sin, cos, tan, pi, e, exp, log, copy, linspace
from sympy import pprint, Symbol
from numpy.polynomial.legendre import leggauss

In [2]:
n_list = [2, 4, 8, 10, 20, 30, 50, 100]
gauss_legendre_n_list = [2, 3, 4, 5, 8, 10, 12, 20, 24, 30, 48, 50, 75, 96, 100, 500, 1000]

In [3]:
x = Symbol('x')
y = Symbol('y')

In [4]:
f1 = lambda x: sin(x)
f2 = lambda x: exp(x) * cos(x)

f1_sol = 2
f2_sol = -(exp(pi) + 1) / 2

In [5]:
trapezios_simples = lambda f, a, b: (b-a)/2 * (f(a) + f(b))
simpson13_simples = lambda f, a, b: (b-a)/3 * (f(a) + 4*f((a + b)/2) + f(b))
simpson38_simples = lambda f, a, b: (3/8)*(b-a) * (f(a) + 3*f((2*a + b)/3) + 3*f((a + 2*b)/3) + f(b))

In [6]:
def trapezios_composta(f, a, b, n):
    h = (b-a)/n
    xi = a
    s_int = 0
    for i in range(n):
        s_int += f(xi) + f(xi+h)
        xi += h
    s_int *= h/2
    return s_int


def simpson13_composta(f, a, b, n):
    h = (b-a)/n
    x = linspace(a, b, n+1)
    s_int = 0
    for i in range(0, n, 2):
        s_int += f(x[i]) + 4*f(x[i+1]) + f(x[i+2])
    s_int *= h/3
    return s_int

In [7]:
from sympy import oo            # símbolo 'infinito'

def gausslegendre(f, a, b, x_pts, w_pts):
    x_gl = copy(x_pts)
    w_gl = copy(w_pts)
    def gl_sum(f, x_list, w_list):
        s_int = 0
        for x, w in zip(x_list, w_list):
            s_int += w * f(x)
        return s_int
    if (a == -1 and b == 1): return gl_sum(f, x_gl, w_gl)
    elif (a == 0 and b == oo):
        x_inf = list(map(lambda x: tan( pi/4 * (1+x)), copy(x_pts)))
        w_inf = list(map(lambda w, x: pi/4 * w/(cos(pi/4 * (1+x)))**2, copy(w_pts), copy(x_pts)))
        return gl_sum(f, x_inf, w_inf)

    else:
        h = (b-a)/2
        xi = list(map(lambda x: h * (x + 1) + a, x_gl))
        return h * gl_sum(f, xi, w_gl)

In [8]:
def gauss_legendre(f, x_pts, w_pts):
    x_gl = copy(x_pts)
    w_gl = copy(w_pts)
    s_int = 0
    for x, w in zip(x_gl, w_gl):
        s_int += w * f(x)
    return s_int
        
def gauss_legendre_ab(f, a, b, x_pts, w_pts):
    w_gl_ab = copy(w_pts)
    x_gl_ab = copy(x_pts)
    h = (b-a)/2
    s_int = 0
    for x, w in zip(x_gl_ab, w_gl_ab):
        s_int += w * f(h * (x + 1) + a)
        s_int *= h
    return s_int
    # x_gl_ab = list(map(lambda x: -1 + 2*(x-a)/(b-a), copy(x_pts)))
    # return gausslegendre(f, x_gl_ab, w_pts)


def gauss_legendre_0inf(f, x_pts, w_pts):
    x_gl_0inf = copy(x_pts)
    w_gl_0inf = copy(w_pts)
    s_int = 0
    for x, w in zip(x_gl_0inf, w_gl_0inf):
        x_gl_0inf = list(map(lambda x: tan( pi/4 * (1+x)), copy(x_pts)))
        w_gl_0inf = list(map(lambda w, x: pi/4 * w/(cos(pi/4 * (1+x)))**2, copy(w_pts), copy(x_pts)))
    return gausslegendre(f, x_gl_0inf, w_gl_0inf)

In [9]:
def erro_rel(est, real): return abs((est-real)/real) * 100

def aval_simples(f, a, b, real_value):
    print('Utilizando o método:')
    trap_si = trapezios_simples(f, a, b)
    print('Trapézio Simples: ' + str(trap_si) + '   Erro Relativo: ' + str(erro_rel(trap_si, real_value)) + ' %')

    simps13_si = simpson13_simples(f, a, b)
    print('Simpson 1/3 Simples: ' + str(simps13_si) + '   Erro Relativo: ' + str(erro_rel(simps13_si, real_value)) + ' %')

    simps38_si =  simpson38_simples(f, a, b)
    print('Simpson 3/8 Simples: ' + str(simps38_si) + '   Erro Relativo: ' + str(erro_rel(simps38_si, real_value)) + ' %')


In [10]:
def aval_composta(f, a, b, n, x_n, w_n, real_value):
    print('Utilizando os métodos: [N = ' + str(n) + '] \n')

    trap_c = trapezios_composta(f, a, b, n)
    print('Trapézios Composta: ' + str(trap_c) + '   Erro Relativo: ' + str(erro_rel(trap_c, real_value)))

    simp_13_c = simpson13_composta(f, a, b, n)
    print('Simpson Composta: ' + str(simp_13_c) + '   Erro Relativo: ' + str(erro_rel(simp_13_c, real_value)))

    gaule_ab = gausslegendre(f, a, b, x_n, w_n)
    print('Gauss-Legendre: ' + str(gaule_ab) + '   Erro Relativo: ' + str(erro_rel(gaule_ab, real_value)))
    print('\n')

Integrando $\sin (x)$

In [11]:
aval_simples(f1, 0, pi, f1_sol)

Utilizando o método:
Trapézio Simples: 1.92367069372e-16   Erro Relativo: 100.0 %
Simpson 1/3 Simples: 4.18879020479   Erro Relativo: 109.439510239 %
Simpson 3/8 Simples: 6.12157285429   Erro Relativo: 206.078642715 %


In [12]:
for n in n_list:
    x_i, w_i = leggauss(n)
    aval_composta(f1, 0, pi, n, x_i, w_i, f1_sol)

Utilizando os métodos: [N = 2] 

Trapézios Composta: 1.57079632679   Erro Relativo: 21.4601836603
Simpson Composta: 2.09439510239   Erro Relativo: 4.71975511966
Gauss-Legendre: 1.93581957465   Erro Relativo: 3.20902126744


Utilizando os métodos: [N = 4] 

Trapézios Composta: 1.89611889794   Erro Relativo: 5.19405510315
Simpson Composta: 2.00455975498   Erro Relativo: 0.227987749221
Gauss-Legendre: 1.99998422846   Erro Relativo: 0.000788577113897


Utilizando os métodos: [N = 8] 

Trapézios Composta: 1.97423160195   Erro Relativo: 1.28841990272
Simpson Composta: 2.00026916995   Erro Relativo: 0.0134584974194
Gauss-Legendre: 2.0   Erro Relativo: 2.77555756156e-13


Utilizando os métodos: [N = 10] 

Trapézios Composta: 1.98352353751   Erro Relativo: 0.823823124527
Simpson Composta: 2.00010951732   Erro Relativo: 0.00547586575022
Gauss-Legendre: 2.0   Erro Relativo: 2.22044604925e-14


Utilizando os métodos: [N = 20] 

Trapézios Composta: 1.99588597271   Erro Relativo: 0.205701364564
Simp

Integrando $e^{x} \cos (x)$

In [13]:
aval_simples(f2, 0, pi, f2_sol)

Utilizando o método:
Trapézio Simples: -34.7785186603   Erro Relativo: 188.131903996 %
Simpson 1/3 Simples: -23.1856791068   Erro Relativo: 92.087935997 %
Simpson 3/8 Simples: -35.3982912992   Erro Relativo: 193.266575551 %


In [14]:
for n in n_list:
    x_i, w_i = leggauss(n)
aval_composta(f2, 0, pi, n, x_i, w_i, f2_sol)

Utilizando os métodos: [N = 100] 

Trapézios Composta: -12.0723318741   Erro Relativo: 0.0164498818044
Simpson Composta: -12.0703460552   Erro Relativo: 2.16413765411e-06
Gauss-Legendre: -12.0703463164   Erro Relativo: 8.68285389478e-13




In [15]:
xi, wi = leggauss(100)

### Calcular o integral: ###

$\int _{0}^{\infty} \frac{x dx}{(1+x)^{4}}$

In [16]:
f3 = lambda x: x / (1+x)**4
gausslegendre(f3, 0, oo, xi, wi)

0.16666666666666732

Usando a transformação

$x = \frac{y}{1-y}$

In [26]:
f3_1 = lambda y: f3(y/(1-y))
print(gausslegendre(f3_1, 0, oo, xi, wi)) 
print(gausslegendre((lambda y: y - y**4), 0, oo, xi, wi))

-4.46366930931e+18
-4.46668290624e+18


E depois a transformação

$x = \tan \left[ \frac{\pi}{4} (1+y) \right]$

In [18]:
f3_2 = lambda y: f3(tan(pi/4 * (1+y)))
gausslegendre(f3_2, 0, oo, xi, wi)

-1865617957364.563

#### Integral Duplo de ####

$\int _{0}^{1} \left( \int _{-\sqrt{1-y^{2}}} ^{\sqrt{1-y^{2}}} \, dx \right) dy$

In [23]:
gausslegendre((lambda y: gausslegendre((lambda x: 1), -(1-y**2)**(1/2), (1-y**2)**(1/2) , xi, wi)), 0, 1, xi, wi)

1.5707966136937224

#### Integral Duplo de ####

$\int _{0}^{1} \left( \int _{-\sqrt{1-y^{2}}} ^{\sqrt{1-y^{2}}} e^{-xy} \, dx \right) dy$

In [21]:
gausslegendre((lambda y: gausslegendre(lambda x: e**(-x*y), -(1-y**2)**(1/2), (1-y**2)**(1/2), xi, wi)), 0, 1, xi, wi)

1.6038298597579663

Derivar $e^{x} \sin(x) + e^{-x} \cos(x)$ nos pontos x = 0, $\frac{\pi}{4}$, $\frac{\pi}{2}$, $\frac{3\pi}{4}$ e $\pi$.

Fórmulas a 2, 3 e 5 pontos.

h = 0.1, 0.05, 0.01

In [44]:
f6 = lambda x: exp(x)*sin(x) + exp(-x)*cos(x)
x_6 = [0, pi/4, pi/2, 3*pi/4, pi]
h_6 = [0.1, 0.05, 0.01]

In [54]:
df_2pts = lambda f, x, h: (f(x+h) - f(x)) / h
df_3pts = lambda f, x, h: (-f(x + 2*h) + 4*f(x+h) - 3*f(x)) / (2*h)
df_5pts = lambda f, x, h: (-3*f(x+4*h) + 16*f(x+3*h) - 36*f(x+2*h) + 48*f(x+h) - 25*f(x)) / (12*h)

In [56]:
for x in x_6:
    pprint('\nDerivada de f(' + str(x) + ') :' )
    for h in h_6:
        print('com passo \'h\' = ' + str(h) + ' :')
        print('Fórmula a 2 pontos: ' + str(df_2pts(f6, x, h)))
        print('Fórmula a 3 pontos: ' + str(df_3pts(f6, x, h)))
        print('Fórmula a 5 pontos: ' + str(df_5pts(f6, x, h)))

                  
Derivada de f(0) :
com passo 'h' = 0.1 :
Fórmula a 2 pontos: 0.106499885754
Fórmula a 3 pontos: -0.0123298081793
Fórmula a 5 pontos: 3.12839432937e-05
com passo 'h' = 0.05 :
Fórmula a 2 pontos: 0.0516458298118
Fórmula a 3 pontos: -0.00320822613034
Fórmula a 5 pontos: 9.08106774962e-07
com passo 'h' = 0.01 :
Fórmula a 2 pontos: 0.0100664999989
Fórmula a 3 pontos: -0.000132333299807
Fórmula a 5 pontos: 2.7154574885e-10
                                   
Derivada de f(0.7853981633974483) :
com passo 'h' = 0.1 :
Fórmula a 2 pontos: 2.6439799125
Fórmula a 3 pontos: 2.45896668342
Fórmula a 5 pontos: 2.45721835016
com passo 'h' = 0.05 :
Fórmula a 2 pontos: 2.55059697185
Fórmula a 3 pontos: 2.45721403119
Fórmula a 5 pontos: 2.45698635082
com passo 'h' = 0.01 :
Fórmula a 2 pontos: 2.4757049983
Fórmula a 3 pontos: 2.45697439475
Fórmula a 5 pontos: 2.4569725301
                                   
Derivada de f(1.5707963267948966) :
com passo 'h' = 0.1 :
Fórmula a 2 pontos: 4.6